In [1]:
using Distributed
addprocs(44);

In [2]:
using pulse_input_DDM

┌ Info: Recompiling stale cache file /usr/people/briandd/.julia/compiled/v1.2/pulse_input_DDM/m8WoA.ji for pulse_input_DDM [c2542da2-1f7d-11e9-2506-87691095ebb2]
└ @ Base loading.jl:1240
  ** incremental compilation may be fatally broken for this module **



In [3]:
f, ncells, ntrials, nparams = "Sigmoid", [5,4], [300,400], 4

θ_syn = θneural(θz = θz(σ2_i = 0.5, B = 40., λ = -0.5, σ2_a = 50., σ2_s = 0.1,
    ϕ = 0.6, τ_ϕ =  0.02),
    θy=[[Sigmoid(10.,60.,0.05*rand([-1,1]),0.) for n in 1:N] for N in ncells], ncells=ncells,
    nparams=nparams, f=f)

θneural{θz{Float64},Array{Array{Sigmoid{Float64},1},1}}
  θz: θz{Float64}
  θy: Array{Array{Sigmoid{Float64},1}}((2,))
  ncells: Array{Int64}((2,)) [5, 4]
  nparams: Int64 4
  f: String "Sigmoid"


In [4]:
data, μ_rnt, μ_t = synthetic_data(θ_syn, ntrials; centered=false);

In [5]:
n = 53;

In [6]:
loglikelihood(θ_syn, data, n)

-182720.58381206868

In [7]:
θy0 = vcat(vcat(initialize_θy.(data, f)...)...);

In [8]:
fit=vcat(trues(dimz), trues(sum(ncells)*nparams))
x0=vcat([0.5, 9., -2., 10., 1., 0.4, 0.008], θy0);

In [9]:
options = neuraloptions(ncells=ncells, x0=x0, fit=fit, nparams=nparams, f=f);

In [ ]:
model, = optimize(data, options, n)

In [29]:
pulse_input_DDM.flatten(model.θ)

10-element Array{Float64,1}:
  0.01              
 40.0               
 -0.9627358924918562
 50.0               
  0.01              
  0.9999999999999998
  0.02              
 10.0               
  9.778879497696998 
 10.0               

In [35]:
pulse_input_DDM.flatten(model.θ)

10-element Array{Float64,1}:
  0.01              
 40.0               
 -0.8310778339064705
 50.0               
  0.01              
  0.9999999999999998
  0.02              
 10.0               
  9.406564860175386 
 10.0               

### Compute Hessian
Blah blah blah

In [22]:
# ### Compute Hessian and the confidence interavls
# Blah blah blah

H = Hessian(model, n)
CI, HPSD = CIs(H);

┌ Warning: Hessian is not positive definite. Approximated by closest PSD matrix.
│             ||ϵ||/||H|| is 3.1634665397929176
└ @ pulse_input_DDM /usr/people/briandd/Projects/pulse_input_DDM/src/base_model.jl:19


In [23]:
CI

10-element Array{Float64,1}:
 0.012932521256961121
 3.497021858319779   
 0.07636794260066583 
 6.2377864707758     
 0.06030334354772982 
 0.009331108991109465
 0.004129265702158947
 0.13748158528160712 
 0.1775739474080006  
 0.6282239304194033  

In [63]:
hcat(collect(pulse_input_DDM.flatten(model.θ)) - CI, collect(pulse_input_DDM.flatten(θ_syn)), collect(pulse_input_DDM.flatten(model.θ)) + CI)

34×3 Array{Float64,2}:
 -7.39695      0.5     8.39695  
  9.35427     40.0    70.6457   
 -1.02275     -0.5     0.0227482
 -7.30294     50.0   107.303    
 -0.952099     0.1     1.1521   
  0.314576     0.6     0.885424 
  0.00727841   0.02    0.0327216
  7.92373     10.0    10.0164   
 -1.23579     -1.0    -0.347883 
  0.123542     0.0     4.17512  
  9.0495      10.0    10.9347   
 -1.35359     -1.0    -0.402627 
 -2.15134      0.0     2.81355  
  ⋮                             
  8.62564     10.0    10.2356   
  0.394949     1.0     1.33324  
 -0.606459     0.0     3.00023  
  9.55002     10.0    11.083    
  0.409178     1.0     1.61034  
 -5.67385      0.0     1.61199  
  9.42011     10.0    10.9106   
 -1.49154     -1.0    -0.453278 
 -3.13858      0.0     1.61907  
  9.44594     10.0    11.2167   
  0.365056     1.0     1.27829  
 -2.59983      0.0     2.66292  

In [64]:
hcat(collect(pulse_input_DDM.flatten(model.θ)), collect(pulse_input_DDM.flatten(θ_syn)))

34×2 Array{Float64,2}:
  0.5         0.5 
 40.0        40.0 
 -0.5        -0.5 
 50.0        50.0 
  0.1         0.1 
  0.6         0.6 
  0.02        0.02
  8.97008    10.0 
 -0.791837   -1.0 
  2.14933     0.0 
  9.99211    10.0 
 -0.878107   -1.0 
  0.331102    0.0 
  ⋮               
  9.43063    10.0 
  0.864096    1.0 
  1.19688     0.0 
 10.3165     10.0 
  1.00976     1.0 
 -2.03093     0.0 
 10.1653     10.0 
 -0.972411   -1.0 
 -0.759752    0.0 
 10.3313     10.0 
  0.821673    1.0 
  0.0315479   0.0 

In [65]:
hcat(collect(pulse_input_DDM.flatten(model.θ)) - CI .< 
    collect(pulse_input_DDM.flatten(θ_syn)) .< 
    collect(pulse_input_DDM.flatten(model.θ)) + CI)

34×1 BitArray{2}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 0
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1